# Hidden Factor Hidden Topics
## Amazon Kindle Store Reviews Dataset
Dataset Link: http://jmcauley.ucsd.edu/data/amazon/
##### Sample Review

In [1]:

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import json

In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
DATASET_SIZE = 30000
DATASET_FILE = '../../Datasets/Apps Dataset/AppReviews30k.json'
TRAIN_SET_FILE = '../../Datasets/Apps Dataset/train_15000.json'
TEST_SET_FILE = '../../Datasets/Apps Dataset/test_5000.json'


In [4]:
dataset_file = open(TRAIN_SET_FILE)
train_set = dataset_file.readlines()

dataset_file = open(TEST_SET_FILE)
test_set = dataset_file.readlines()

print len(train_set)
print len(test_set)

15000
5000


In [5]:
data = []
for line in train_set:
    data.append(json.loads(line)["reviewText"])
print data[0]

My son plays this game all the time. He loves Umizoomi. He has all the badges and trophies for the first two levels on here, but the others will not unlock. I don't know how many times you have to replay the same two games, but now he's had it for months and is sick of it because it's just to long to wait for something else to happen.


In [6]:
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                stop_words='english',
                                token_pattern=r'\b[a-zA-Z]{3,}\b',
                                max_df=0.5,
                                min_df=5
                               )
dtm_tf = tf_vectorizer.fit_transform(data)

In [7]:
print (tf_vectorizer.get_feature_names()[:10])
print (dtm_tf.shape)

[u'abc', u'abilities', u'ability', u'able', u'absolute', u'absolutely', u'accept', u'acceptable', u'access', u'accessed']
(15000, 4154)


In [8]:
NUMBER_OF_TOPICS = 5


lda_tf = LatentDirichletAllocation(n_components=NUMBER_OF_TOPICS, random_state=18)
lda_tf.fit(dtm_tf)

C:\Users\chris\Anaconda2\envs\cs670\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=5, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=18,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [9]:


ReviewsByUser = {}
ItemReviews = {}
AverageRatingsByUsers = {}
AverageRatingsByItems = {}
UserHistory = {}
GlobalAverage = 0.0
for line in train_set:
    d = json.loads(line)
    user = d["reviewerID"]
    item = d["asin"]
    try: 
        ReviewsByUser[user] += ' ' + d["reviewText"]
    except:
        ReviewsByUser[user] = d["reviewText"]
    try: 
        ItemReviews[item] += ' ' + d["reviewText"]
    except:
        ItemReviews[item] = d["reviewText"]
    try:
        AverageRatingsByUsers[user] = (AverageRatingsByUsers[user][0]+float(d["overall"]), AverageRatingsByUsers[user][1]+1)
    except:
        AverageRatingsByUsers[user] = (float(d["overall"]), 1)
    try:
        AverageRatingsByItems[item] = (AverageRatingsByItems[item][0]+float(d["overall"]), AverageRatingsByItems[item][1]+1)
    except:
        AverageRatingsByItems[item] = (float(d["overall"]), 1)
    try:
        UserHistory[user].append(item) 
    except:
        UserHistory[user] = [item]
    
    GlobalAverage += float(d["overall"])

GlobalAverage /= len(train_set)

for k in AverageRatingsByUsers.keys():
    AverageRatingsByUsers[k] = AverageRatingsByUsers[k][0]*1.0/AverageRatingsByUsers[k][1]        
        
for k in AverageRatingsByItems.keys():
    AverageRatingsByItems[k] = AverageRatingsByItems[k][0]*1.0/AverageRatingsByItems[k][1]

# print AverageRatingsByItems
# print AverageRatingsByUsers
print GlobalAverage
print "done"

4.05813333333
done


In [10]:
import numpy as np

WordToTopicMapping = {}

tf_feature_names = tf_vectorizer.get_feature_names()

rows = NUMBER_OF_TOPICS # topics count
columns = len(tf_feature_names) # Total number of words
    
M = np.zeros((rows, columns))
total_of_columns = np.sum(lda_tf.components_, axis=0)
print len(total_of_columns)
for i in range(columns):
    column_sum = total_of_columns[i]
    for j in range(rows):
        M[j][i] = lda_tf.components_[j][i]*1.0/column_sum

M = np.transpose(M)
DocVecRep = CountVectorizer(vocabulary=tf_feature_names) # document vector representation

4154


In [12]:
# print M

def MakeProbabilityDistribution(Gamma):
    column_sums = np.sum(Gamma, axis=0)
    return np.divide(Gamma, 1.0*column_sums)

Gamma_i = np.zeros((len(ItemReviews.keys()), NUMBER_OF_TOPICS)) #   #items   X  topics

j = 0
# Calculating latent factors for items
ItemToIndexMapping = {}
for item in ItemReviews.keys():
    ItemToIndexMapping[item] = j
    review = ItemReviews[item]
    review_vector = DocVecRep.fit_transform([review]).toarray()
    Gamma_i[j] = np.matmul(review_vector, M)
    j += 1


Gamma_i = np.transpose(Gamma_i)
Gamma_i = MakeProbabilityDistribution(Gamma_i)
print Gamma_i.shape

Gamma_u = np.zeros((len(ReviewsByUser.keys()), NUMBER_OF_TOPICS))
j = 0
# Calculating latent factors for users
UserToIndexMapping = {}
for user in ReviewsByUser.keys():
    UserToIndexMapping[user] = j
    review = ReviewsByUser[user]
    review_vector = DocVecRep.fit_transform([review]).toarray()
    Gamma_u[j] = np.matmul(review_vector, M)
    j += 1

Gamma_u = np.transpose(Gamma_u)
Gamma_u = MakeProbabilityDistribution(Gamma_u)
Gamma_u = np.transpose(Gamma_u)
print Gamma_u.shape

# print UserToIndexMapping
# print ItemToIndexMapping

    

(5L, 1842L)
(3139L, 5L)


In [13]:

total_users = len(ReviewsByUser.keys())
total_items = len(ItemReviews.keys())

ActualRatingAvailability = np.zeros((total_users, total_items))

for line in train_set:
    d = json.loads(line)
    userId = d["reviewerID"]
    itemId = d["asin"]
    user = UserToIndexMapping[userId]
    item = ItemToIndexMapping[itemId]

In [14]:
def PredictUserToItemRating(userId, itemId):
    userIndex = UserToIndexMapping[userId]
    itemIndex = ItemToIndexMapping[itemId]
    gamma_user = Gamma_u[userIndex]
    gamma_item = Gamma_i[:, itemIndex]
    return np.dot(gamma_user, gamma_item)

def IsActuallyRated(userId, itemId):
    user = UserToIndexMapping[userId]
    item = ItemToIndexMapping[itemId]
    
    return ActualRatingAvailability[user][item] == 1

RECOMMENDATIONS = 5

def RecommendItemsToUser(userId):
    itemsList = ItemToIndexMapping.keys()
    predictions = []
    for item in itemsList:
        if not IsActuallyRated(userId, item):
            predictions.append((item, PredictUserToItemRating(userId, item)))
    sorted_by_predictions = sorted(predictions, key=lambda tup: tup[1], reverse=True)
    print len(sorted_by_predictions)
    print sorted_by_predictions[:min(RECOMMENDATIONS, len(predictions))]
    return sorted_by_predictions[:min(RECOMMENDATIONS, len(predictions))]

def GetRecommendationsForUser(userId):
    predictions = RecommendItemsToUser(userId)
    return [_[0] for _ in predictions]

cnt = 0
def GetEvaluationOnTestData():
    y_pred = []
    y_actual = []
    for line in test_set:
        d = json.loads(line)
        userId = d["reviewerID"]
        itemId = d["asin"]
        #print PredictUserToItemRating(userId, itemId)

        try:
            prediction = AverageRatingsByUsers[userId] + AverageRatingsByItems[itemId] - GlobalAverage + PredictUserToItemRating(userId, itemId)
            y_pred.append(prediction)
            y_actual.append(d["overall"])
        except:
            cnt += 1
            #print cnt,
            pass
            
            
    MAE = round(mean_absolute_error(y_actual, y_pred), 2)
    RMSE = round(np.sqrt(mean_squared_error(y_actual, y_pred)), 2)
    return (MAE, RMSE)

USER_ID = 'A00100742Q4O8VH0YMUBZ'
print GetEvaluationOnTestData()
history = UserHistory[USER_ID]
recommendations = GetRecommendationsForUser(USER_ID)
print recommendations
print history
print cnt

(1.03, 1.4)
1842
[(u'B0097C8XL0', 0.2462655906964141), (u'B00FDUH4WY', 0.24606602162718505), (u'B00IG63BVK', 0.2426768072507653), (u'B00821NZ74', 0.2418034337266109), (u'B00FPYJLCY', 0.2410069405531767)]
[u'B0097C8XL0', u'B00FDUH4WY', u'B00IG63BVK', u'B00821NZ74', u'B00FPYJLCY']
[u'B00ANT8OF6', u'B00AMR1HZ8', u'B004WGGQPQ', u'B006C1ZSO4', u'B008LY1B32', u'B007KPT2N4', u'B0080JJLBW', u'B009P8EMCK']
0


In [15]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

C:\Users\chris\Anaconda2\envs\cs670\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      28.999920        1       1 -0.132669 -0.076110
3      24.324081        1       2  0.121562 -0.127035
2      17.693707        1       3 -0.136806 -0.168210
1      14.766524        1       4  0.311718  0.090769
4      14.215768        1       5 -0.163805  0.280586, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
2162  Default  3534.000000       love  3534.000000  30.0000  30.0000
160   Default  5483.000000        app  5483.000000  29.0000  29.0000
1515  Default  4648.000000        fun  4648.000000  28.0000  28.0000
2687  Default  3733.000000       play  3733.000000  27.0000  27.0000
2004  Default  2280.000000     kindle  2280.000000  26.0000  26.0000
249   Default   756.000000    awesome   756.000000  25.0000  25.0000
1536  Default  2307.000000      games  2307.000000  24.0000  24.0000
2894  Default   708.000000       quot   708.000000  23.0000  23.0000
1630  Default   974.000000   graphics   974.000000  22.0000  22.0000
2969  Default   973.000000  recommend   973.000000  21.0000  21.0000
2488  Default   587.000000        old   587.000000  20.0000  20.0000
1634  Default  3018.000000      great  3018.000000  19.0000  19.0000
2325  Default   827.000000      money   827.000000  18.0000  18.0000
1594  Default   512.000000      going   512.000000  17.0000  17.0000
2941  Default  2598.000000     really  2598.000000  16.0000  16.0000
3710  Default  1077.000000      think  1077.000000  15.0000  15.0000
2081  Default   715.000000      level   715.000000  14.0000  14.0000
1503  Default   478.000000    friends   478.000000  13.0000  13.0000
800   Default   471.000000       cool   471.000000  12.0000  12.0000
180   Default   584.000000       apps   584.000000  11.0000  11.0000
4099  Default   773.000000      works   773.000000  10.0000  10.0000
2694  Default  1603.000000    playing  1603.000000   9.0000   9.0000
1732  Default   394.000000       help   394.000000   8.0000   8.0000
1995  Default   639.000000       kids   639.000000   7.0000   7.0000
1342  Default   616.000000        far   616.000000   6.0000   6.0000
3749  Default   513.000000      times   513.000000   5.0000   5.0000
2090  Default  4601.000000       like  4601.000000   4.0000   4.0000
1004  Default   526.000000        did   526.000000   3.0000   3.0000
488   Default   804.000000        buy   804.000000   2.0000   2.0000
3479  Default   351.000000      start   351.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
138    Topic5   216.935668      angry   217.734722   1.9471  -5.0604
988    Topic5   209.793522  developer   210.615239   1.9469  -5.0939
1736   Topic5   178.345543      helps   179.148855   1.9463  -5.2563
3670   Topic5   178.451355       tell   179.258724   1.9463  -5.2557
1074   Topic5   177.575705      doing   178.387177   1.9463  -5.2606
1702   Topic5   171.513589       hate   172.324255   1.9461  -5.2953
3541   Topic5   169.278989      stuck   170.084264   1.9461  -5.3084
1206   Topic5   164.947991  enjoyable   165.751867   1.9460  -5.3344
3293   Topic5   164.730208      short   165.539880   1.9459  -5.3357
2918   Topic5   164.891049     rating   165.703549   1.9459  -5.3347
3205   Topic5   151.444168     search   152.254319   1.9455  -5.4198
3321   Topic5   151.397323    similar   152.207909   1.9455  -5.4201
910    Topic5   147.594428   daughter   148.396386   1.9454  -5.4455
611    Topic5   146.358130   children   147.160635   1.9454  -5.4539
1952   Topic5   190.298631      items   191.378340   1.9452  -5.1914
3711   Topic5   138.544276   thinking   139.348145   1.9450  -5.5088
2707   Topic5   203.141818       plus   205.082224   1.9413  -5.1261
1386   Topic5   251.354506     figure   304.435451   1.7592  -4.9131
2081   Topic5   400.687175      level   715.322365   1.3713 

In [17]:
import json

# Converting json to votes file format
DATASET_FILE = '../../Datasets/Apps Dataset/Apps20k.json'
dataset_file = open(DATASET_FILE)
dataset_lines = dataset_file.readlines()

output_file = open('../../Datasets/Apps Dataset/Apps20k.votes', 'w')
for line in dataset_lines:
    d = json.loads(line)
    output_file.write(d["reviewerID"] + ' ' + d["asin"]+ ' ' + str(d["overall"]) + ' ' + str(d["unixReviewTime"]) + ' ' + str(len(d["reviewText"].split())) + ' ' + d["reviewText"] + '\n')
output_file.close()

In [18]:
def GetURL(products_list):
    print products_list
    products = ','.join([str(_) for _ in products_list])
    url = "http://www.asinlab.com/php/convertfromasin.php?asin_num=" + products + "&id_type=UPC&bulk=true&x=false"
    return url
    
print GetURL(recommendations)
print GetURL(history)

[u'B0097C8XL0', u'B00FDUH4WY', u'B00IG63BVK', u'B00821NZ74', u'B00FPYJLCY']
http://www.asinlab.com/php/convertfromasin.php?asin_num=B0097C8XL0,B00FDUH4WY,B00IG63BVK,B00821NZ74,B00FPYJLCY&id_type=UPC&bulk=true&x=false
[u'B00ANT8OF6', u'B00AMR1HZ8', u'B004WGGQPQ', u'B006C1ZSO4', u'B008LY1B32', u'B007KPT2N4', u'B0080JJLBW', u'B009P8EMCK']
http://www.asinlab.com/php/convertfromasin.php?asin_num=B00ANT8OF6,B00AMR1HZ8,B004WGGQPQ,B006C1ZSO4,B008LY1B32,B007KPT2N4,B0080JJLBW,B009P8EMCK&id_type=UPC&bulk=true&x=false


In [ ]:
DATASET_FILE = '../../Datasets/Apps Dataset/AppReviewsFull.json'
dataset_file = open(DATASET_FILE)
dataset_lines = dataset_file.readlines()[:30000]

output_file = open('../../Datasets/Apps Dataset/AppReviews30k.json', 'w')
for line in dataset_lines:
    output_file.write(line.strip() + '\n')
output_file.close()